<a href="https://colab.research.google.com/github/Henil21/Mental_health_NLP/blob/main/Mental_health.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd

In [15]:
Data=pd.read_csv('/content/drive/MyDrive/mental_health.csv')
Data.count()

text     27977
label    27977
dtype: int64

In [16]:
text=Data['text']
label=Data['label']
label.value_counts()


0    14139
1    13838
Name: label, dtype: int64

In [17]:
train_shf=Data.sample(frac=1,random_state=42)
train_shf.head()

,text,label
6604,im ready gothanks previous person commented la...,1
25912,atoms ashim posting im anonymous molested old...,1
18308,wanna use omegle cus self esteems low ppl alwa...,0
18875,hey world questioni curious think lot people r...,1
23181,guuuuuys passed dance exams honors got honors ...,0


In [18]:
# lets visualize some random training example
import random
random_index=random.randint(0, len(train_shf)-5)
# create random index not higher than total number of samples
for row in train_shf[["text","label"]][random_index:random_index+5].itertuples():
  _,text,label=row
  print(f"label:{label}","(mental health abnormal)" if label>0 else "(normal)")
  print(f"Text:\n{text}\n")
  print("----\n")

label:0 (normal)
Text:
anyone wanna exchange music im looking new music listen to like rap music not cap mumble rap stuff actual good singers good beat want new songs playlist pm want talk

----

label:1 (mental health abnormal)
Text:
homeless third timethey kidding said life s would hard ive homeless  different times  separate occasions currently age  struggle major drugs smoke lot weedwax find drinking often slip depression also lot often figured id comment advice improve try get situation recently lost job sick  months ago took  days one week sick cold want risk getting anyone else sick especially peak covid emailed company  different times letting know situation  months later guess matter fired it explain further need explain companies policies used work unimportant whats happening now fact is lost job due poor communication skills relaying im sick never get job back im paying it  months later completely unexpected nowhere waking morning phone call hr representative terminated almo

In [19]:
from sklearn.model_selection import train_test_split

train_sentences,val_sentences,train_labels,val_labels=train_test_split(Data["text"].to_numpy(),
                                                                       Data["label"].to_numpy(),
                                                                       test_size=0.3,
                                                                       random_state=42)


In [20]:
train_sentences[:10]

array(['silly putty implies serious putty',
       'literally sat bed crying begging someone something kill lmaoi feel fuckin trapped mani one want worry shit thanks reading guess method  would done tonight',
       'lot delayed this im doneim sorry thank kind words cant keep anymore cant',
       'im going  hours get worki couldnt get much sleep thing going head wanna die wanna die wanna die over could succeeded six redflag attempts ive made past year',
       'hi help guy told would put blenderleave blended mean',
       'plop join fast among us ehejidjdjdjdjdjdidjdjddjd dbbd',
       'perfect man alive  correct answer kim taehyung',
       'happy wife happy life haha kidding ill never happy life wife miserable ha joshin you im gay',
       'racist anything racist fuck marathons like bruh cmon aint one got need god damn marathon',
       'yeahi really much left except alcohol absolutly one life turn times like this always anyone else world remember things tell them im rational want c

In [22]:
train_labels[:10]

array([0, 1, 1, 1, 0, 0, 0, 0, 0, 1])

In [24]:
# find the average number of token(words) in the training tweet
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

72

In [25]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 72 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                     split="whitespace",
                                    output_mode="int",
                                    output_sequence_length=max_length)


In [28]:
text_vectorizer.adapt(train_sentences)
sample_text='this is stupid someone kill me fuck'
text_vectorizer([sample_text])

<tf.Tensor: shape=(1, 72), dtype=int64, numpy=
array([[ 68, 256, 252,  40,  69,  10, 124,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0]])>

In [32]:
import random
random_sentence=random.choice(train_sentences)
random_sentence
print(f'orignal text is\n {random_sentence} \n\n and the vectorized text is \n {text_vectorizer([random_sentence])}')

orignal text is
 tonight hopefullyif fucking penny every time ive convinced would jeff bezos hopefully tonight night dont want keep holding back pussying out want really commit time 

 and the vectorized text is 
 [[ 504    1   43 5619   49   15   12 1415   14 2939    1 1105  504  166
    23    4  104 1016   39    1  142    4   17  591   15    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]]


In [33]:
words_in_voc=text_vectorizer.get_vocabulary()
top_5=words_in_voc[:5]
bottom_5=words_in_voc[-5:]
print(f"number of word in vocab {len(words_in_voc)}\n")
print(f"5 most comman word in  vocab {top_5}\n")
print(f"5 least comman  word in vocab {bottom_5}")


number of word in vocab 10000

5 most comman word in  vocab ['', '[UNK]', 'im', 'like', 'want']

5 least comman  word in vocab ['botched', 'bosses', 'boomers', 'boo', 'boiled']


In [34]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,#set the input shape
                             output_dim=128,
                             input_length=max_length)

In [35]:
# Convert text into number
from sklearn.feature_extraction.text import TfidfVectorizer
# our model
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline

# create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),# convert words to number using tfidf
    ("clf",MultinomialNB())# model the text
])
# fit the pipleine to the training data
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [36]:
baseline_score=model_0.score(val_sentences,val_labels)
# as we use .evaluate in tf for sklearn its .score
baseline_score*100

83.50011913271385

In [37]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
def calculate_results(y_true,y_pred):
  # Calculate model accuracy
  model_accuracy=accuracy_score(y_true,y_pred)*100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision":model_precision*100,
                   "recall":model_recall*100,
                   "f1":model_f1*100
                   }
  return model_results

In [41]:
from tensorflow.keras import layers
import tensorflow as tf
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
# x=layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x) #layers.Bidirectional works with any LSTM
# print(x.shape)
x=layers.Bidirectional(layers.GRU(64))(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_4=tf.keras.Model(inputs,outputs,name="model_4_bidirectional")

In [42]:
# Compile model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [43]:
# Fit the model
model_4_history=model_4.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels))

Epoch 1/5
612/612 [==============================] - 54s 82ms/step - loss: 0.2700 - accuracy: 0.8845 - val_loss: 0.1977 - val_accuracy: 0.9238
Epoch 2/5
612/612 [==============================] - 49s 81ms/step - loss: 0.1320 - accuracy: 0.9495 - val_loss: 0.2072 - val_accuracy: 0.9192
Epoch 3/5
612/612 [==============================] - 49s 80ms/step - loss: 0.0802 - accuracy: 0.9690 - val_loss: 0.2495 - val_accuracy: 0.9165
Epoch 4/5
612/612 [==============================] - 50s 81ms/step - loss: 0.0518 - accuracy: 0.9812 - val_loss: 0.2888 - val_accuracy: 0.9188
Epoch 5/5
612/612 [==============================] - 48s 79ms/step - loss: 0.0320 - accuracy: 0.9889 - val_loss: 0.3492 - val_accuracy: 0.9108


In [45]:
y_pred=model_4.predict(val_sentences)
y_pred[:10]

263/263 [==============================] - 4s 14ms/step


array([[9.9996513e-01],
       [9.9962193e-01],
       [6.9211833e-02],
       [9.9977595e-01],
       [8.9636138e-03],
       [9.9992478e-01],
       [9.9992013e-01],
       [9.9997479e-01],
       [9.5078790e-01],
       [1.3668947e-04]], dtype=float32)

AttributeError: ignored

In [46]:
model_preds=tf.squeeze(tf.round(y_pred))
model_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 0., 1., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [49]:
calculate_results(y_true=val_labels,
                            y_pred=model_preds)

{'accuracy': 91.07695973314273,
 'precision': 91.2232201322854,
 'recall': 91.07695973314273,
 'f1': 91.0749147278404}

In [93]:
import numpy as np
import math
text_s=['homeless third timethey kidding said life s would hard ive homeless  different times  separate occasions currently age  struggle major drugs smoke lot weedwax find drinking often slip depression also']
txt=np.array(text_s)
y_pred=model_4.predict(txt)s
# model_preds=tf.squeeze(tf.round(y_pred))
print(int(tf.round(y_pred)))


1/1 [==============================] - 0s 21ms/step
1
